# Music Data Modeling Project
In this project, we apply Data Modeling with Cassandra and build an ETL pipeline using Python. We will build a Data Model around our queries that we want to get answers for. For our use case we want below answers:
1. Get details of a song that was herad on the music app history during a particular session.
2. Get songs played by a user during particular session on music app.
3. Get all users from the music app history who listened to a particular song.

# Part I. ETL Pipeline for Pre-Processing the Files

In [4]:
# import python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [5]:
# process original event csv data files
# check current working directory
print(f"current working directory: {os.getcwd()}")
# get directory of the data files
filepath = os.getcwd() + '/event_data'
# create a list of the files and collect each filepath
for root,dirs,files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    filepath_list = glob.glob(os.path.join(root,'*'))
    print(filepath_list)

current working directory: /Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling
['/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-15-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-22-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-09-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-18-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-04-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_Data_Modeling/event_data/2018-11-01-events.csv', '/Users/caseyddd/Library/CloudStorage/Dropbox/Baimax/PersonalDevelopment/SQL/Music_D

# Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
full_data_rows_list = []

for fp in filepath_list:
    with open(fp,'r',encoding='utf8',newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for l in csvreader:
            full_data_rows_list.append(l)

print(f'total rows: {len(full_data_rows_list)}')
print(f'sample data:\n {full_data_rows_list[:5]}')

total rows: 8056
sample data:
 [['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26'], ['Train', 'Logged In', 'Ryan', 'M', '2', 'Smith', '205.45261', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Marry Me', '200', '1.54224E+12', '26'], ['', 'Logged In', 'Wyatt', 'M', '0', 'Scott', '', 'free', 'Eureka-Arcata-Fortuna, CA', 'GET', 'Home', '1.54087E+12', '563', '', '200', '1.54225E+12', '9'], ['', 'Logged In', 'Austin', 'M', '0', 'Rosales', '', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'GET', 'Home', '1.54106E+12', '521', '', '200', '1.54225E+12', '12']]
